# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import time
import csv
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.models import model_from_json

# Settings

In [2]:
# Path
image_path = '../02_Data/face_age' # Added path to gitingnore, you will have to add data to this path

# Image Manipulation

In [3]:
def load_data(folder_path, img_size=(200, 200)):
    X = []
    y = []
    for folder in os.listdir(folder_path):
        if os.path.isdir(os.path.join(folder_path, folder)):
            age = int(folder)
            for file in os.listdir(os.path.join(folder_path, folder)):
                img_path = os.path.join(folder_path, folder, file)
                img = Image.open(img_path)
                img = img.resize(img_size)
                img = np.array(img)
                X.append(img)
                y.append(age)
    return np.array(X), np.array(y)

folder_path = image_path
img_size = (200, 200)
X, y = load_data(folder_path, img_size)

In [4]:
print(
    f"Shape of X: {X.shape}", "\n",
    f"- {X.shape[0]}: Number of images in the dataset", "\n",
    f"- {X.shape[1]}: Height of each image", "\n",
    f"- {X.shape[2]}: Width of each image", "\n",
    f"- {X.shape[3]}: Number of channels of each image (Red, Green, and Blue)"
    )

Shape of X: (9778, 200, 200, 3) 
 - 9778: Number of images in the dataset 
 - 200: Height of each image 
 - 200: Width of each image 
 - 3: Number of channels of each image (Red, Green, and Blue)


In [5]:
print(f"Labels: {y}")

Labels: [  1   1   1 ... 101 110 110]


In [6]:
# Normalize pixel values
X = X / 255.0

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
print(
    f"Size of arrays:", "\n",
    f"- X_train shape: {X_train.shape}", "\n",
    f"- X_test shape: {X_test.shape}", "\n",
    f"- y_train shape: {y_train.shape}", "\n",
    f"- y_test shape: {y_test.shape}"
    )

Size of arrays: 
 - X_train shape: (7822, 200, 200, 3) 
 - X_test shape: (1956, 200, 200, 3) 
 - y_train shape: (7822,) 
 - y_test shape: (1956,)


In [8]:
optimizer = "adam"
loss = "mean_squared_error"
metrics = ['mae']


def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (5, 5), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(256, (3, 3), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1)
    ])

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

input_shape = (img_size[0], img_size[1], 3)
model = create_model(input_shape)

In [9]:
# Train the model
epochs = 1000
batch_size = 16
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/1000
489/489 [==============================] - 26s 35ms/step - loss: 935.9125 - mae: 24.6942 - val_loss: 781.6656 - val_mae: 23.1869
Epoch 2/1000
489/489 [==============================] - 14s 29ms/step - loss: 288.4002 - mae: 12.9678 - val_loss: 174.8781 - val_mae: 9.5067
Epoch 3/1000
489/489 [==============================] - 15s 30ms/step - loss: 189.3459 - mae: 10.3354 - val_loss: 110.8776 - val_mae: 7.5183
Epoch 4/1000
489/489 [==============================] - 15s 30ms/step - loss: 169.0360 - mae: 9.7430 - val_loss: 100.1402 - val_mae: 7.3854
Epoch 5/1000
489/489 [==============================] - 13s 27ms/step - loss: 161.7164 - mae: 9.4485 - val_loss: 114.0884 - val_mae: 7.9139
Epoch 6/1000
489/489 [==============================] - 14s 30ms/step - loss: 145.7326 - mae: 8.8815 - val_loss: 101.3391 - val_mae: 7.0285
Epoch 7/1000
489/489 [==============================] - 14s 28ms/step - loss: 145.5965 - mae: 8.9583 - val_loss: 89.0942 - val_mae: 6.7216
Epoch 8/1000
489/

# Save, Load, and Display Model History

In [ ]:
def append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae):
    train_mae = round(train_mae, 6)
    validation_mae = round(validation_mae, 6)

    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Model', 'Input Shape', 'Epochs', 'Batch Size', 'Optimizer', 'Loss Function', 'Train MAE', 'Validation MAE'])

    with open(file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        try:
            next(csv_reader)  # Skip the header row
        except StopIteration:
            pass  # The CSV file is empty or only contains the header row

        for row in csv_reader:
            if list(map(str, row[1:])) == list(map(str, [input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])):
                print("Entry with the same parameters already exists.")
                return False

    with open(file_name, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])

    return True

timestamp = int(time.time())
file_name = '../04_Age_Prediction/model_history.csv'
description = f"{timestamp}"
input_shape = str(X_train.shape[1:])
optimizer = optimizer
loss_function = loss
train_mae = history.history['mae'][-1]
validation_mae = history.history['val_mae'][-1]

new_entry_added = append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae)

# Save model
def save_model_architecture(model, file_name):
    model_json = model.to_json()
    with open(file_name, "w") as json_file:
        json_file.write(model_json)

if new_entry_added:
    model_architecture_file = f"../04_Age_Prediction/models/{description}.json"
    save_model_architecture(model, model_architecture_file)
else:
    print("Model not saved as an entry with the same parameters already exists.")


In [ ]:
def load_data_to_dataframe(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None
    
    df = pd.read_csv(file_name)
    return df

file_name = '../04_Age_Prediction/model_history.csv'
df = load_data_to_dataframe(file_name)

df

In [ ]:
# Specify model to load
model_to_load = "1683270923"

def load_and_display_model_architecture(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None

    with open(file_name, "r") as json_file:
        model_json = json_file.read()
    
    model = model_from_json(model_json)
    model.summary()

    return model

model_architecture_file = f"../04_Age_Prediction/models/{model_to_load}.json"
model = load_and_display_model_architecture(model_architecture_file)

# 